In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/pre_data"

test  train_preliminary


In [0]:
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install xgboost
# !pip install matplotlib
# !pip install keras
# !pip install sklearn

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers.embeddings import Embedding
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

Using TensorFlow backend.


Found GPU at: /device:GPU:0


In [0]:
df1 = pd.read_csv("/content/drive/My Drive/pre_data/train_preliminary/user.csv")
df1 = pd.read_csv("/content/drive/My Drive/pre_data/train_preliminary/user.csv")
df2 = pd.read_csv("/content/drive/My Drive/pre_data/train_preliminary/ad.csv")
df4 = pd.read_csv("/content/drive/My Drive/pre_data/test/ad.csv")
df2 = pd.concat([df2, df4])
df2.sort_values('creative_id', inplace=True)
df2 = df2.drop_duplicates(subset=['creative_id'], keep='first')
df3 = pd.read_csv("/content/drive/My Drive/pre_data/train_preliminary/click_log.csv")
df5 = pd.read_csv("/content/drive/My Drive/pre_data/test/click_log.csv")
df3 = pd.concat([df3, df5])
df = pd.merge(df3, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.sort_values(['user_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
63668278,4000000,75,3596158,1,3096233,18,36668
63668279,4000000,75,3642395,1,3135640,2,18422
63668280,4000000,76,366858,1,331268,2,36890
63668281,4000000,76,3333680,1,2868147,2,32830


In [0]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(5000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X = df[['user_id', 'time', 'click_times']]
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(4000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(70000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, x_tmp], axis=1)
X

(63668300, 1)
636683/636683 [==============================] - 16s 25us/step
(63668300, 1)
636683/636683 [==============================] - 16s 25us/step
(63668300, 1)
636683/636683 [==============================] - 16s 25us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,0.029431,-0.033295,-0.048701,0.020023,-0.031545,-0.004856,0.007321,0.025283,0.040615,-0.015532,-0.029778,0.023860,-0.039823,-0.039710,0.021950,-0.016358,-0.031450,0.006530,-0.007577,0.000131
1,1,20,1,-0.048235,-0.005688,0.024030,0.043112,-0.003828,-0.010483,-0.016740,0.015189,-0.035309,-0.046667,-0.010862,-0.042516,-0.035942,0.010851,0.029017,0.044061,-0.026044,-0.009893,-0.047110,0.004164
2,1,20,1,0.042290,0.023681,-0.019502,0.029208,-0.028854,-0.022953,0.017174,-0.000901,-0.018921,0.035398,-0.029778,0.023860,-0.039823,-0.039710,0.021950,-0.014499,0.030393,-0.013623,-0.047796,-0.048044
3,1,39,1,0.036444,-0.011700,-0.025014,-0.010694,-0.007201,-0.029231,-0.009614,-0.000625,-0.021594,0.030811,-0.029778,0.023860,-0.039823,-0.039710,0.021950,0.030552,0.020115,-0.045190,-0.037754,-0.046111
4,1,40,1,-0.031125,0.009703,0.010178,0.034072,-0.029927,0.031847,-0.016728,0.046624,0.026689,-0.043569,-0.010862,-0.042516,-0.035942,0.010851,0.029017,0.041795,0.005284,0.023822,-0.026186,0.027513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63668278,4000000,75,1,-0.029270,0.038458,-0.045154,0.024458,-0.046926,-0.045021,-0.028880,-0.031036,-0.045071,-0.032521,-0.016586,0.002515,0.018743,-0.020553,0.037505,0.026567,-0.006920,0.047487,0.044049,-0.015232
63668279,4000000,75,1,0.011127,0.042764,-0.031359,-0.026650,0.046490,0.044724,-0.009285,-0.011985,-0.017965,0.030415,-0.010862,-0.042516,-0.035942,0.010851,0.029017,0.046578,-0.031834,-0.037357,0.041098,-0.017937
63668280,4000000,76,1,-0.029682,-0.025320,-0.024607,-0.029493,0.011337,-0.024033,-0.013852,-0.038404,0.017355,0.026998,-0.010862,-0.042516,-0.035942,0.010851,0.029017,0.007808,-0.005264,-0.008751,-0.010884,0.003772
63668281,4000000,76,1,-0.037959,-0.024530,-0.011062,0.043673,-0.033979,0.041287,-0.034084,0.046680,-0.023889,-0.032552,-0.010862,-0.042516,-0.035942,0.010851,0.029017,0.026248,-0.026888,0.001546,-0.026843,-0.025328


In [0]:
x_tmp = None
y_tmp = None
df2 = None
df3 = None
df4 = None
df5 = None
X['time'] = X['time'] / 91
X['click_times'] = X['click_times'] / 152

TypeError: ignored

In [0]:
X_tmp = X.iloc[:,0]
X_tmp

0                 1
1                 1
2                 1
3                 1
4                 1
             ...   
63668278    4000000
63668279    4000000
63668280    4000000
63668281    4000000
63668282    4000000
Name: user_id, Length: 63668283, dtype: int64

In [0]:
X_tmp = X_tmp.drop_duplicates(keep='last')


KeyError: ignored

In [0]:
X_tmp = X_tmp.reset_index()
X_tmp = X_tmp['index']
X_tmp

0                12
1                57
2                87
3               116
4               149
             ...   
1899995    63668100
1899996    63668120
1899997    63668139
1899998    63668219
1899999    63668282
Name: index, Length: 1900000, dtype: int64

In [0]:
from tensorflow.keras.utils import Sequence
class DataGenerater(Sequence):
    def __init__(self, batch_size, X, total, start_id, y_test, x_index):
        self.X = X
        self.batch_size = batch_size
        self.total = total
        self.start_id = start_id
        self.y_test = np.array(y_test)
        self.x_index = x_index
    def _check_matrix(self, x):
        if x.shape[0] >= 30:
            return x.iloc[:30, :]
        else:
            tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
            tmp1 = pd.DataFrame(tmp1)
            tmp1.columns = x.columns
            return pd.concat([x, tmp1], axis=0)
    def __getitem__(self, index):
        a = self.start_id + self.batch_size * index
        b = self.start_id + self.batch_size * (index + 1) - 1
        if index == 0:
            Y = self.X[:self.x_index.iloc[b] + 1]
        else:
            Y = self.X[self.x_index.iloc[a - 1] + 1:self.x_index.iloc[b] + 1]
        Y = Y.groupby('user_id').apply(self._check_matrix)
        Y = Y.reset_index(drop=True)
        Y = Y.drop('user_id', axis = 1)
        return np.array(Y).reshape((-1, 30, 22)), self.y_test[self.batch_size * index:self.batch_size * (index + 1)]
    def __len__(self):
        return self.total // self.batch_size

In [0]:
train_generator = DataGenerater(batch_size=16, X=X, total=900000, start_id=0, y_test=df1.iloc[:900000, 2:3], 
                               x_index=X_tmp.iloc[:900000])
model = models.Sequential()
model.add(layers.Masking(mask_value=0., input_shape=(30, 22)))
model.add(layers.LSTM(32))
model.add(layers.Dense(1))
model.summary()
model.compile(loss='MSE', optimizer='adam')
model.fit_generator(train_generator, epochs=10)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, 30, 22)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                7040      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 7,073
Trainable params: 7,073
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 4175/56250 [=>............................] - ETA: 34:21 - loss: 0.2273

KeyboardInterrupt: ignored

In [0]:
Y2_predict = model.predict(X_test)
Y2_predict = np.round(Y2_predict)
ans = 0 
for i in range(100000):
    if Y2_predict[i] == df1.iloc[i + 800000, 2]:
        ans += 1
print('accurency:', ans / 100000)

accurency: 0.823


In [0]:
Y1_train, Y1_test = df1.iloc[:800000, 1:2], df1.iloc[800000:, 1:2]
Y1_train = pd.get_dummies(Y1_train['age'])
model2 = models.Sequential()
model2.add(layers.Masking(mask_value=0., input_shape=(20, 22)))
model2.add(layers.LSTM(32))
model2.add(layers.Dense(20, activation = 'relu'))
model2.add(layers.Dense(10, activation = 'softmax'))
model2.summary()
model2.compile(loss='MSE', optimizer='adam')
model2.fit(X_train, Y1_train, epochs = 100, batch_size=32)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 20, 22)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                7040      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                660       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 7,910
Trainable params: 7,910
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25000/25000 [==============================] - 99s 4ms/step - loss: 0.0846
Epoch 2/100
25000/25000 [==============================] - 97s 4ms/step - loss: 0.0838
Epoch 3/100
25000/25000 [==============================] - 97s 4ms/step - lo

In [0]:
Y1_predict = model.predict(X_test)
ans = 0 
for i in range(100000):
    if np.argmax(Y1_predict[i]) == df1.iloc[i + 800000, 1] - 1:
        ans += 1
print('accurency:', ans / 100000)